In [36]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install pytorch_lightning

In [37]:
import pandas as pd
from google.colab import drive
import numpy as np
import math
from tqdm import tqdm
from datetime import date, timedelta
import torch
from collections import OrderedDict
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import time

In [38]:
from pytorch_lightning import LightningModule


In [39]:
from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
import matplotlib.pyplot as plt

# New Section

In [40]:
drive.mount('/content/drive')
path = '/content/drive/My Drive/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
file1 = 'la_weather_with_pm_per_day.csv'
weather_data = pd.read_csv(path+file1)
file2 = 'metadata_with_station.csv'
metadata = pd.read_csv(path+file2)
metadata = metadata[metadata['location'] == 'Los Angeles (SoCAB)'].reset_index(drop = True)

In [42]:
def data_to_numpy(weather_data, edge_names, node_names, station_ids, date_range): 

    graph_node_features = np.empty((len(date_range), len(station_ids), len(node_names)))
    graph_edge_features = np.empty((len(date_range), len(station_ids), len(edge_names)))
    graph_labels = np.empty((len(date_range), len(station_ids)))

    for day_idx in range(len(date_range)): 
        for station_idx in range(len(station_ids)): 
                d = date_range[day_idx]             #get date from index
                station = station_ids[station_idx]     #get station number from index
                vals = weather_data[weather_data['DATE'] == d]  #get data date using date
                vals = vals[vals['STATION'] == station]         #get data of station on date
                pm = vals['pm25'].values
                edge = vals[edge_names]
                edge_vals = np.array(edge.values.tolist()).flatten()  #edge feature as array
                node_vals = vals[node_names]
                node_vals = np.array(node_vals.values.tolist()).flatten() #node features as array
                if len(node_vals) == 0:                           #if there is no weather data on date, set to all zeros
                    node_vals = np.zeros(len(node_names))
                graph_node_features[day_idx, station_idx] = node_vals 
                if len(pm) == 0:     #if no pm label on date set to 0
                    pm = np.zeros(1)
                graph_labels[day_idx, station_idx] = pm
                if len(edge_vals) == 0:  #if no edge feature, set to 0
                    edge_vals = np.zeros(len(edge_names))
                graph_edge_features[day_idx, station_idx] = edge_vals

    return graph_node_features, graph_edge_features, graph_labels

def generate_nodes(metadata, stations):
    stations.sort()
    nodes = OrderedDict()
    for id in stations:
        row = metadata[metadata['STATION'] == id]
        lat = row['Latitudes'].values[0]
        lon = row['Longitudes'].values[0]
        # elev = row['Elevetation']
        nodes.update({id:{'Latitude':lat,'Longitude':lon}})
    return nodes

def geo_distance(first_node, second_node):
        '''Haversine formula for geodesic distance'''
        lat1, long1 = first_node
        lat2, long2 = second_node
        R = 6371e3 #m
        lat1 = lat1 * np.pi/180; #rad
        lat2 = lat2 * np.pi/180; #rad
        delta_lat = (lat2 - lat1) * np.pi/180;
        delta_long = (long1 - long2) * np.pi/180;
        a = (np.sin(delta_lat/2))**2 + np.cos(lat1) * np.cos(lat2) * np.sin(delta_long/2) * np.sin(delta_long/2)
        c = 2 * np.arctan2(a**0.5, (1-a)**0.5)
        d = R * c #m
        return d

def generate_node_distances(coordinates):
    distance_matrix = np.zeros((len(coordinates), len(coordinates)))
    for i in range(len(coordinates)): 
        coord1 = coordinates[i]
        for j in range(len(coordinates)): 
            coord2 = coordinates[j]
            distance = geo_distance(coord1, coord2)
            distance_matrix[i][j] = distance
    return distance_matrix

def sparse_adjacency(adj): 
    """Converts a dense adjacency matrix to a sparse adjacency matrix defined
    by edge indices and edge attributes.
    """
    assert adj.dim() >= 2 and adj.dim() <= 3
    assert adj.size(-1) == adj.size(-2)

    index = adj.nonzero(as_tuple=True)
    edge_attr = adj[index]

    if len(index) == 3:
        batch = index[0] * adj.size(-1)
        index = (batch + index[1], batch + index[2])

    return torch.stack(index, dim=0), edge_attr

def add_self_loops_to_sparse_adj(edge_idx, n):
        source_w_self_loop = np.append(edge_idx[0], [i for i in range(n)])    #add self loops
        target_w_self_loop = np.append(edge_idx[1], [i for i in range(n)])
        edge_idx = np.array([source_w_self_loop, target_w_self_loop])
        order = edge_idx[0].argsort()
        edge_idx[0].sort()
        edge_idx[1] = edge_idx[1][order] 
        return(edge_idx) 

class Graph():
    def __init__(self,
                 graph_metadata,
                 stations,
                 edge_data, 
                 distance_threshold):
        self.graph_metadata = graph_metadata[graph_metadata['STATION'].isin(stations)]
        self.distance_threshold = distance_threshold
        self.nodes = generate_nodes(graph_metadata, stations)
        self.size = len(self.nodes)
        self.edge_data = edge_data
        self.edge_index, self.edge_attr = self.generate_edges()
        self.adjacency_matrix = self.get_adjacency_matrix()
        self.edge_adjacency = self.get_edge_adjacency()
        self.transformation_matrix = self.get_transformation_matrix()

    def generate_edges(self):
        nodes = self.nodes            
        node_list = list(self.nodes)        #get list of node ids
        coordinates = list(zip(self.graph_metadata['Latitudes'], self.graph_metadata['Longitudes'])) 
        distance_matrix = generate_node_distances(coordinates)  #square matrix of distance to all other nodes
        adj_matrix = np.zeros([self.size, self.size])
        adj_matrix[distance_matrix < self.distance_threshold] = 1 #in adj matrix, set entry to 1 if distance between nodes below threshold

        distance_matrix = distance_matrix * adj_matrix
        mean_distance = np.mean(distance_matrix)
        std_distance = np.std(distance_matrix)

        edge_idx, edge_dist = sparse_adjacency(torch.tensor(distance_matrix)) #edge_idx : shape (2 * number of connected nodes (dis < threshold)) 
        edge_idx, edge_dist = edge_idx.numpy(), edge_dist.numpy()             #edge_dist: same shape as above, distance values between those node indices
        edge_idx = add_self_loops_to_sparse_adj(edge_idx, len(node_list))
        edge_idx = add_self_loops_to_sparse_adj(edge_idx, len(node_list))

        windx, windy, dx, dy = [],[],[],[]
        for i in range(edge_idx.shape[1]):
            #get index of non-zero edges
            source_idx = edge_idx[0, i]
            dest_idx = edge_idx[1, i]
            #get lat lon for the nodes at ends of non zero edge
            key0 = node_list[source_idx]
            lat0 = nodes[key0]['Latitude']
            long0 = nodes[key0]['Longitude']
            key1 = node_list[dest_idx]
            lat1 = nodes[key1]['Latitude']
            long1 = nodes[key1]['Longitude']
            distance_x = geo_distance((lat0, 0), (lat1, 0))
            distance_y = geo_distance((0, long1), (0, long1))
            edge_vect_x = distance_x 
            #get wind along x and y vectors from both source and edge
            wind_source_x = self.edge_data[:, source_idx, 0]
            wind_dest_x = self.edge_data[:, dest_idx, 0]
            wind_source_y = self.edge_data[:, source_idx, 1]
            wind_dest_y = self.edge_data[:, dest_idx, 1]
            #average source and edge wind components to get net wind 
            wind_x = (wind_source_x + wind_dest_x)/2
            wind_y = (wind_source_y + wind_dest_y)/2
            mean_wind_x, mean_wind_y = np.mean(wind_x), np.mean(wind_y)
            std_wind_x, std_wind_y = np.std(wind_x), np.std(wind_y)
            wind_x = (wind_x - mean_wind_x) / std_wind_x
            wind_y = (wind_y - mean_wind_y) / std_wind_y
            distance_x = (distance_x - mean_distance) / std_distance
            distance_x = (distance_y - mean_distance) / std_distance

            # edge_vectors.append(np.array([distance_x, distance_y, wind_x, wind_y]))
            windx.append(wind_x), windy.append(wind_y)
            dx.append(distance_x), dy.append(distance_y)
        windx = np.stack(windx).transpose()
        windy = np.stack(windy).transpose()
        dx = np.tile(np.stack(dx), [windx.shape[0],1])
        dy = np.tile(np.stack(dy), [windx.shape[0],1])
        edge_vectors = np.array([windx, windy, dx, dy]).transpose(1,2,0)
        return edge_idx, edge_vectors

    def edge_list_to_adj(self):
        adj = np.identity(self.edge_index.size)
        for k,(i,j) in enumerate(zip(self.edge_index[0], self.edge_index[1])):
            adj[i,j] = self.edge_attr[j]
        return adj

    def edge_list_sequence_to_adj(self):
        adjacencies = []
        for i in range(self.edge_attr.shape[0]):
            adj = np.identity(self.size)
            for k,(i,j) in enumerate(zip(self.edge_index[0], self.edge_index[1])):
                adj[i,j] = self.edge_attr[i][k]
            adjacencies.append(adj)
        return np.array(adjacencies)

    def get_edge_adjacency(self):
        adj_e = np.zeros([self.edge_index.shape[1],self.edge_index.shape[1]])
        for i,s in enumerate(self.edge_index[0]):
            t = self.edge_index[1][i]
            adj_e[i,t] = 1
        assert adj_e.all() == adj_e.transpose().all()
        return adj_e

    def get_adjacency_matrix(self):
        nodes = self.nodes            
        node_list = list(self.nodes)        #get list of node ids
        coordinates = list(zip(self.graph_metadata['Latitudes'], self.graph_metadata['Longitudes']))  #zip latitude and longitude of all nodes
        distance_matrix = generate_node_distances(coordinates)  #square matrix of distance to all other nodes

        adj = np.identity(self.size)
        adj[distance_matrix < self.distance_threshold] = 1 #in adj matrix, set entry to 1 if distance between nodes below threshold
        assert adj.all() == adj.transpose().all()
        return adj

    def get_transformation_matrix(self):
        T = np.zeros([self.size, self.edge_adjacency.shape[0]])
        for i,source in enumerate(self.edge_index[0]):
            for j,t in enumerate(self.edge_index[1]): 
                if self.edge_index[0][j]  == source:        
                    T[source, t] = 1      
        assert T.all() == T.transpose().all()
        return T



In [43]:
stations_2018_02 = weather_data[weather_data['DATE'] == '2018-02-01']['STATION'].unique() 
stations_2018_06 = weather_data[weather_data['DATE'] == '2018-06-08']['STATION'].unique() 
stations_2020_01 = weather_data[weather_data['DATE'] == '2020-01-03']['STATION'].unique() 
stations_2020_12 = weather_data[weather_data['DATE'] == '2020-12-31']['STATION'].unique() 


stations = [value for value in stations_2018_02 if value in stations_2018_06]

In [44]:
start = date(2018,2,1)
end = date(2021,6,8)
date_range = pd.date_range(start,end-timedelta(days=1))
date_range = [str(x)[:10] for x in date_range]
node_cols = ['ceiling', 'visibility', 'dew', 'precipitation_duration', 'precipitation_depth', 'mean_aod','min_aod','max_aod']
edge_cols = ['wind_x', 'wind_y']
stations.sort()
weather_data = weather_data[weather_data['STATION'].isin(stations)]
weather_data = weather_data[weather_data['DATE'].isin(date_range)]
weather_data
weather_data = weather_data[['STATION','DATE','pm25']+edge_cols+node_cols]
weather_data = weather_data.fillna(weather_data.mean())


graph_node_features, graph_edge_features, graph_labels = data_to_numpy(weather_data, edge_cols, node_cols, stations, date_range)
print(graph_node_features.shape)
print(graph_edge_features.shape)
print(graph_labels.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  if sys.path[0] == '':


(1223, 9, 8)
(1223, 9, 2)
(1223, 9)


In [45]:
graph = Graph(metadata, stations, graph_edge_features, distance_threshold = 30e3)


In [46]:
graph.adjacency_matrix.shape

(9, 9)

In [47]:
def get_sliding_timeframe(vect, sequence_len):
    vect_len = vect.shape[0]
    timeframes = []
    for i in range(sequence_len, vect_len):
        timeframe = vect[i - sequence_len : i]
        timeframes.append(timeframe)
    timeframes = np.stack(timeframes, axis = 0)
    return timeframes

def normalize(vect, mean, std):
    norm_v = (vect - mean) / std
    return norm_v



class WeatherData(Dataset):
    def __init__(self, 
                 labels, 
                 edge_features,   
                 node_features,
                 historical_len, 
                 prediction_len, 
                 sequence = True):
        
        self.historical_len = historical_len
        self.prediction_len = prediction_len
        self.sequence_len = historical_len + prediction_len

        self.edge_features = edge_features

        label_mean = labels.mean(axis = (0,1))
        label_sdev = labels.std(axis = (0,1))
        self.labels = normalize(labels, label_mean, label_sdev)

        feature_mean = node_features.mean()
        feature_sdev = node_features.std()
        self.features = normalize(node_features, feature_mean, feature_sdev)

        if sequence: 
            self.features = get_sliding_timeframe(self.features, self.sequence_len)
            self.edge_features = get_sliding_timeframe(self.edge_features, self.sequence_len)
            self.labels = get_sliding_timeframe(self.labels, self.sequence_len)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.features[idx], self.edge_features[idx], self.labels[idx]
        return data

In [48]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features_v, out_features_v, in_features_e, out_features_e, bias=True, node_layer=True):
        super(GraphConvolution, self).__init__()
        self.in_features_e = in_features_e
        self.out_features_e = out_features_e
        self.in_features_v = in_features_v
        self.out_features_v = out_features_v

        if node_layer:
            self.node_layer = True
            self.weight = Parameter(torch.FloatTensor(in_features_v, out_features_v))
            self.p = Parameter(torch.from_numpy(np.random.normal(size=(1, in_features_e))).float())
            if bias:
                self.bias = Parameter(torch.FloatTensor(out_features_v))
            else:
                self.register_parameter('bias', None)
        else:
            self.node_layer = False
            self.weight = Parameter(torch.FloatTensor(in_features_e, out_features_e))
            self.p = Parameter(torch.from_numpy(np.random.normal(size=(1, in_features_v))).float())
            if bias:
                self.bias = Parameter(torch.FloatTensor(out_features_e))
            else:
                self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, H_v, H_e, adj_e, adj_v, T):
        if self.node_layer:
            multiplier1 = torch.mm(T, torch.diag((H_e @ self.p.t()).t()[0])) @ T.t()
            mask1 = torch.eye(multiplier1.shape[0])
            M1 = mask1 * torch.ones(multiplier1.shape[0]) + (1. - mask1)*multiplier1
            adjusted_A = torch.mul(M1, adj_v)
            '''
            print("adjusted_A is ", adjusted_A)
            normalized_adjusted_A = adjusted_A / adjusted_A.max(0, keepdim=True)[0]
            print("normalized adjusted A is ", normalized_adjusted_A)
            '''
            # to avoid missing feature's influence, we don't normalize the A
            output = torch.mm(adjusted_A, torch.mm(H_v, self.weight))
            if self.bias is not None:
                ret = output + self.bias
            return ret, H_e

        else:
            multiplier2 = torch.spmm(T.t(), torch.diag((H_v @ self.p.t()).t()[0])) @ T
            mask2 = torch.eye(multiplier2.shape[0])
            M3 = mask2 * torch.ones(multiplier2.shape[0]) + (1. - mask2)*multiplier2
            adjusted_A = torch.mul(M3, adj_e)
            normalized_adjusted_A = adjusted_A / adjusted_A.max(0, keepdim=True)[0]
            output = torch.mm(adjusted_A.float(), torch.mm(H_e, self.weight).float())
            if self.bias is not None:
                ret = output + self.bias
            else: 
                ret = output
            return H_v, ret


        
class GCN(nn.Module):
    def __init__(self, nfeat_v, nfeat_e, nhid, nclass, dropout):
        super(GCN, self).__init__()
 
        self.gc1 = GraphConvolution(nfeat_v, nhid, nfeat_e, nfeat_e, node_layer=True).to(device)
        self.gc2 = GraphConvolution(nhid, nhid, nfeat_e, nfeat_e, node_layer=False).to(device)
        self.gc3 = GraphConvolution(nhid, nclass, nfeat_e, nfeat_e, node_layer=True).to(device)
        self.dropout = dropout

    def forward(self, X, Z, adj_e, adj_v, T):
        # print x 
        gc1 = self.gc1(X, Z, adj_e, adj_v, T)
        X, Z = F.relu(gc1[0]), F.relu(gc1[1])
        
        X = F.dropout(X, self.dropout, training=self.training)
        Z = F.dropout(Z, self.dropout, training=self.training)

        gc2 = self.gc2(X, Z, adj_e, adj_v, T)
        X, Z = F.relu(gc2[0]), F.relu(gc2[1])


        X = F.dropout(X, self.dropout, training=self.training)
        Z = F.dropout(Z, self.dropout, training=self.training)

        X, Z = self.gc3(X, Z, adj_e, adj_v,T)
        return F.log_softmax(X, dim=1)

In [49]:
node_features = torch.from_numpy(graph_node_features)
edge_features = torch.from_numpy(graph.edge_attr)
labels = torch.from_numpy(graph_labels)

dataset = WeatherData(labels = labels,
                      edge_features = edge_features,
                      node_features = node_features,
                      historical_len = 5,
                      prediction_len = 2, 
                      sequence = False)

dataloader = DataLoader(dataset, batch_size = 1)
for data in dataloader:
    node_features, edge_features, labels = data
    # graph_data = Data(node_features, edge_index, edge_features, labels)
    break
    

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device = 'cpu'

model = GCN(nfeat_v = 8, nfeat_e = 4, nhid = 4, nclass = 1, dropout = 0.6)
model = model.to(device)


node_features = node_features.to(device).float().squeeze()
edge_features = edge_features.to(device).float().squeeze()
adj_e = adj_e = torch.from_numpy(graph.edge_adjacency)
adj_e.to(device).float().squeeze()
adj = torch.from_numpy(graph.adjacency_matrix)
adj = adj.to(device).float().squeeze()
T = torch.from_numpy(graph.transformation_matrix)
T = T.to(device).float().squeeze()


In [74]:
output[0][0]

tensor(0., grad_fn=<SelectBackward0>)

In [67]:
labels.shape

torch.Size([1, 9])

In [84]:
criteria = torch.nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
idx_train = [0,1,4,6,7,8]

for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    output = model(node_features, edge_features, adj_e, adj, T)
    train_labels = labels[0][idx_train]
    train_output = output[idx_train].squeeze()
    loss_train = criteria(train_output, train_labels)

    print(output)

    loss_train.backward()
    optimizer.step()

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<LogSoftmaxBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<LogSoftmaxBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<LogSoftmaxBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<LogSoftmaxBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<LogSoftmaxBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<LogSoftmaxBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [0

In [ ]:
output[idx_train]

In [ ]:
adj_e

In [ ]:
weather_data = weather_data.fillna(weather_data.mean())